# Setup

Make sure you have already tried `basics.ipynb`, then follow the cells and instructions below.

In [ ]:
from pybricksdev.connections import BLEPUPConnection, EV3Connection
from asyncio import gather, sleep

# TODO: Move something like this to Pybricksdev
class BLEPUPConnectionExtra(BLEPUPConnection):
    def line_handler(self, line):
        try:
            self.line_buffer.append(line)
        except:
            self.line_buffer = []
        super().line_handler(line)

try:
    tilt, car
except:
    car = BLEPUPConnectionExtra()
    await car.connect('90:84:2B:4A:2B:75')
    
    tilt = BLEPUPConnectionExtra()
    await tilt.connect('90:84:2B:0A:15:6C')

### This script will run on the hub with the tilt sensor

For now we just save it. We will run it later.

In [ ]:
%%file build/remote.py

from pybricks.pupdevices import TiltSensor
from pybricks.parameters import Port
from pybricks.tools import wait, StopWatch

imu = TiltSensor(Port.A)

wait(1000)

while True:
    print("__data__{0}".format(imu.tilt()))
    wait(100)

### This script will run on the vehicle robot


For now we just save it. We will run it later.

In [ ]:
%%file build/car.py

from pybricks.pupdevices import Motor
from pybricks.parameters import Port, Direction
from pybricks.tools import wait, StopWatch

left = Motor(Port.B, Direction.COUNTERCLOCKWISE)
right = Motor(Port.A)

while True:
    pitch, roll = eval(input())
    
    left.dc(pitch + roll)
    right.dc(pitch - roll)

### This will run on the PC
For now we just save the function. We will run it later.

In [ ]:
# We can do anything we want with the data, like displaying it.
# In this example, we just send everything received from one hub
# to all the other hubs.

DATA = b'__data__'

async def forwarder(*hubs):
    await sleep(2)
    
    while any(hub.state == hub.RUNNING for hub in hubs):
        for hub in hubs:
            while len(hub.line_buffer) > 0:
                line = hub.line_buffer.pop(0)
                if DATA in line:
                    for remote in hubs:
                        if remote != hub:  
                            await remote.write(line[len(DATA):] + b'\r\n')
        await sleep(0.1)

### Run everything

This runs both robot scripts and the PC script at the same time.

In [ ]:
await gather(
    tilt.run('build/remote.py'),
    car.run('build/car.py'),
    forwarder(tilt, car)
)